In [1]:
!pip install selenium


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.chrome.options import Options
import re

class IndeedJobScraper:
    def __init__(self, headless=False):
        """Initialize the scraper with Chrome options"""
        chrome_options = Options()
        if headless:
            chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument('--disable-blink-features=AutomationControlled')
        chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

        self.driver = webdriver.Chrome(options=chrome_options)
        self.wait = WebDriverWait(self.driver, 10)
        self.job_data = []
        self.jobs_by_category = {}  # Track jobs collected per category

    def search_jobs(self, designations, num_jobs_per_designation):
        """Search for jobs on Indeed across multiple designations

        Args:
            designations: List of job designations to search for
            num_jobs_per_designation: Number of jobs to collect per designation
        """
        # Initialize tracking for each designation
        for designation in designations:
            self.jobs_by_category[designation] = 0

        print(f"\n{'='*60}")
        print(f"Starting multi-category scraping...")
        print(f"Target: {num_jobs_per_designation} jobs per designation")
        print(f"{'='*60}\n")

        # Continue scraping until all categories are filled
        max_pages = 50  # Safety limit
        page = 0

        while not self.all_categories_filled(designations, num_jobs_per_designation) and page < max_pages:
            page += 1

            # Cycle through each designation to find jobs
            for designation in designations:
                # Check if this category already has enough jobs
                if self.jobs_by_category[designation] >= num_jobs_per_designation:
                    continue

                jobs_needed = num_jobs_per_designation - self.jobs_by_category[designation]
                print(f"\n--- Searching for {designation} (Need {jobs_needed} more) ---")

                # Search for this designation
                self.scrape_designation_page(designation, designations, num_jobs_per_designation, page)

                time.sleep(random.uniform(3, 5))

            # Show progress
            self.show_progress(designations, num_jobs_per_designation)

        print(f"\n{'='*60}")
        print(f"Scraping completed!")
        print(f"{'='*60}\n")

        return sum(self.jobs_by_category.values())

    def scrape_designation_page(self, current_designation, all_designations, target_per_category, page_num):
        """Scrape a single page for a specific designation"""
        base_url = "https://www.indeed.com/jobs"
        start = page_num * 10

        # Construct search URL
        search_url = f"{base_url}?q={current_designation.replace(' ', '+')}&l=United States&start={start}"

        try:
            self.driver.get(search_url)
            time.sleep(random.uniform(2, 4))

            # Find all job cards
            job_cards = self.driver.find_elements(By.CSS_SELECTOR, 'div.job_seen_beacon')

            if not job_cards:
                print(f"  No more jobs found on page {page_num}")
                return

            for card in job_cards:
                # Check if all categories are filled
                if self.all_categories_filled(all_designations, target_per_category):
                    break

                try:
                    # Click on job card
                    card.click()
                    time.sleep(random.uniform(1, 2))

                    # Extract job details (without pre-assigned category)
                    job_info = self.extract_job_details()

                    if job_info:
                        # Intelligently categorize the job based on its title
                        matched_category = self.match_job_to_category(job_info['job_title'], all_designations)

                        if matched_category and self.jobs_by_category[matched_category] < target_per_category:
                            job_info['search_category'] = matched_category

                            # Validate job data
                            validation_result = self.validate_job_data(job_info)

                            if validation_result['valid']:
                                self.job_data.append(job_info)
                                self.jobs_by_category[matched_category] += 1
                                salary_display = f"${job_info['salary']}" if job_info['salary'] != 'Not specified' else 'No salary info'
                                print(f"  ✓ [{matched_category}] {self.jobs_by_category[matched_category]}/{target_per_category} - {job_info['job_title']} at {job_info['company_name']} ({salary_display})")
                            else:
                                print(f"  ✗ Skipped - {validation_result['reason']}")
                        else:
                            if matched_category:
                                print(f"  ⊗ [{matched_category}] already filled - skipping")
                            else:
                                print(f"  ⊗ No category match - skipping")

                except Exception as e:
                    print(f"  Error processing job card: {str(e)}")
                    continue

        except Exception as e:
            print(f"  Error loading page: {str(e)}")

    def match_job_to_category(self, job_title, designations):
        """Match a job title to one of the target designations

        Returns the best matching designation or None
        """
        if not job_title or job_title == 'Not specified':
            return None

        job_title_lower = job_title.lower()

        # Create matching keywords for each designation
        designation_keywords = {}
        for designation in designations:
            # Extract key terms from designation
            terms = designation.lower().split()
            designation_keywords[designation] = terms

        # Score each designation based on keyword matches
        scores = {}
        for designation, keywords in designation_keywords.items():
            score = sum(1 for keyword in keywords if keyword in job_title_lower)
            if score > 0:
                scores[designation] = score

        # Return designation with highest score
        if scores:
            return max(scores, key=scores.get)

        return None

    def all_categories_filled(self, designations, target):
        """Check if all categories have reached their target"""
        return all(self.jobs_by_category.get(des, 0) >= target for des in designations)

    def show_progress(self, designations, target):
        """Display progress for all categories"""
        print(f"\n{'='*60}")
        print("PROGRESS REPORT:")
        for designation in designations:
            count = self.jobs_by_category.get(designation, 0)
            percentage = (count / target) * 100
            bar = '█' * int(percentage / 5) + '░' * (20 - int(percentage / 5))
            print(f"  {designation}: [{bar}] {count}/{target} ({percentage:.1f}%)")
        print(f"{'='*60}\n")

    def extract_job_details(self):
        """Extract detailed information from job posting"""
        job_info = {
            'search_category': '',  # Will be assigned later based on matching
            'job_title': '',
            'company_name': '',
            'requirements_and_qualifications': '',
            'responsibilities': '',
            'salary': 'Not specified',
            'experience_years': 'Not specified'
        }

        try:
            # Job Title (Exact designation including levels)
            try:
                job_title = self.driver.find_element(By.CSS_SELECTOR, 'h1.jobsearch-JobInfoHeader-title')
                job_info['job_title'] = job_title.text.strip()
            except:
                try:
                    job_title = self.driver.find_element(By.CSS_SELECTOR, '[data-testid="jobsearch-JobInfoHeader-title"]')
                    job_info['job_title'] = job_title.text.strip()
                except:
                    job_info['job_title'] = 'Not specified'

            # Company Name
            try:
                company = self.driver.find_element(By.CSS_SELECTOR, '[data-testid="inlineHeader-companyName"]')
                job_info['company_name'] = company.text.strip()
            except:
                try:
                    company = self.driver.find_element(By.CSS_SELECTOR, '.jobsearch-InlineCompanyRating')
                    job_info['company_name'] = company.text.split('\n')[0].strip()
                except:
                    job_info['company_name'] = 'Not specified'

            # Salary - Check multiple locations
            salary_found = False

            # Method 1: Check the salary header element
            try:
                salary = self.driver.find_element(By.CSS_SELECTOR, '[data-testid="jobsearch-jobDescriptionHeader-salaryDescription"]')
                job_info['salary'] = salary.text.strip()
                salary_found = True
            except:
                pass

            # Method 2: Check for salary in the job attributes section
            if not salary_found:
                try:
                    salary = self.driver.find_element(By.CSS_SELECTOR, '#salaryInfoAndJobType')
                    salary_text = salary.text.strip()
                    if salary_text and ('a year' in salary_text.lower() or 'an hour' in salary_text.lower() or '$' in salary_text):
                        job_info['salary'] = salary_text.split('\n')[0].strip()
                        salary_found = True
                except:
                    pass

            # Method 3: Check salary guide/info sections
            if not salary_found:
                try:
                    salary_elements = self.driver.find_elements(By.CSS_SELECTOR, '[class*="salary"], [class*="Salary"]')
                    for element in salary_elements:
                        text = element.text.strip()
                        if text and ('$' in text or 'year' in text.lower() or 'hour' in text.lower()):
                            job_info['salary'] = text.split('\n')[0].strip()
                            salary_found = True
                            break
                except:
                    pass

            # Method 4: Extract from job description text
            if not salary_found:
                try:
                    description = self.driver.find_element(By.ID, 'jobDescriptionText')
                    salary_from_description = self.extract_salary_from_text(description.text)
                    if salary_from_description:
                        job_info['salary'] = salary_from_description
                        salary_found = True
                except:
                    pass

            # Job Description (contains requirements, responsibilities, etc.)
            try:
                description = self.driver.find_element(By.ID, 'jobDescriptionText')
                full_text = description.text.lower()

                # Extract Requirements and Qualifications (combined)
                requirements = self.extract_section(full_text,
                    ['requirement', 'qualification', 'required skills', 'must have', 'eligibility',
                     'education', 'degree', 'bachelor', 'master', 'experience', 'skills'])
                job_info['requirements_and_qualifications'] = requirements if requirements else 'Not specified'

                # Extract Responsibilities
                responsibilities = self.extract_section(full_text,
                    ['responsibilit', 'duties', 'you will', 'role', 'job duties'])
                job_info['responsibilities'] = responsibilities if responsibilities else 'Not specified'

                # Extract Experience
                experience = self.extract_experience(full_text)
                if experience:
                    job_info['experience_years'] = experience

            except Exception as e:
                print(f"  Error extracting description: {str(e)}")

        except Exception as e:
            print(f"  Error in extract_job_details: {str(e)}")

        return job_info

    def extract_section(self, text, keywords):
        """Extract specific sections from job description"""
        lines = text.split('\n')
        section_lines = []
        capturing = False

        for i, line in enumerate(lines):
            # Check if line contains any keyword
            if any(keyword in line.lower() for keyword in keywords):
                capturing = True
                section_lines.append(line)
                # Get next several lines
                for j in range(i+1, min(i+10, len(lines))):
                    if lines[j].strip() and not any(stop in lines[j].lower()
                        for stop in ['about', 'company', 'benefits', 'equal opportunity']):
                        section_lines.append(lines[j])
                    else:
                        break
                break

        return ' '.join(section_lines).strip()[:800] if section_lines else ''

    def extract_experience(self, text):
        """Extract years of experience from text"""
        patterns = [
            r'(\d+)\+?\s*(?:years?|yrs?)\s*(?:of)?\s*experience',
            r'experience[:\s]+(\d+)\+?\s*(?:years?|yrs?)',
            r'minimum\s*(?:of)?\s*(\d+)\s*(?:years?|yrs?)'
        ]

        for pattern in patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                return f"{match.group(1)}+ years"

        return 'Not specified'

    def extract_salary_from_text(self, text):
        """Extract salary information from job description text"""
        # Common salary patterns
        patterns = [
            r'\$[\d,]+\.?\d*\s*[-–to]+\s*\$[\d,]+\.?\d*\s*(?:per|/)?\s*(?:year|yr|annum|hour|hr)?',  # $80,000 - $100,000 per year
            r'\$[\d,]+\.?\d*[kK]\s*[-–to]+\s*\$[\d,]+\.?\d*[kK]\s*(?:per|/)?\s*(?:year|yr|annum)?',  # $80K - $100K
            r'\$[\d,]+\.?\d*\s*(?:per|/)?\s*(?:year|yr|annum|hour|hr)',  # $80,000 per year
            r'salary\s*:?\s*\$[\d,]+\.?\d*[kK]?',  # Salary: $80,000
            r'compensation\s*:?\s*\$[\d,]+\.?\d*[kK]?',  # Compensation: $80K
        ]

        lines = text.split('\n')
        for line in lines[:30]:  # Check first 30 lines
            for pattern in patterns:
                match = re.search(pattern, line, re.IGNORECASE)
                if match:
                    salary_text = match.group(0).strip()
                    # Clean up the salary text
                    if len(salary_text) > 5 and len(salary_text) < 100:
                        return salary_text

        return None

    def validate_job_data(self, job_info):
        """Validate if job data meets quality and eligibility criteria

        Returns dict with 'valid' (bool) and 'reason' (str) keys
        """
        # Fields to check for data quality (excluding search_category and job_title)
        fields_to_check = [
            'company_name',
            'requirements_and_qualifications',
            'responsibilities',
            'salary',
            'experience_years'
        ]

        # Count how many fields are "Not specified"
        not_specified_count = sum(
            1 for field in fields_to_check
            if job_info.get(field, '').strip() == 'Not specified' or not job_info.get(field, '').strip()
        )

        # Check if more than 2 fields are missing
        if not_specified_count > 2:
            return {'valid': False, 'reason': f'Insufficient data ({not_specified_count} fields missing, max 2 allowed)'}

        # Check if Master's degree is in requirements/qualifications
        requirements_qual = job_info.get('requirements_and_qualifications', '').lower()

        # If requirements/qualifications is "Not specified", reject it
        if requirements_qual == 'not specified' or not requirements_qual.strip():
            return {'valid': False, 'reason': 'Requirements and qualifications not specified'}

        # Check if "master" or "masters" is mentioned in requirements/qualifications
        has_masters = any(term in requirements_qual for term in [
            'master', 'masters', 'ms ', 'm.s.', 'graduate degree', 'advanced degree'
        ])

        if not has_masters:
            return {'valid': False, 'reason': "Master's degree not in requirements"}

        return {'valid': True, 'reason': ''}

    def is_valid_job_data(self, job_info):
        """Legacy method - kept for backward compatibility"""
        result = self.validate_job_data(job_info)
        return result['valid']

    def save_to_csv(self, filename='indeed_jobs.csv'):
        """Save scraped data to CSV"""
        if self.job_data:
            df = pd.DataFrame(self.job_data)
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            print(f"\n{'='*60}")
            print(f"Data saved to {filename}")
            print(f"Total jobs scraped: {len(self.job_data)}")

            # Show data quality statistics
            print(f"\nData Quality Report:")
            print(f"  All jobs include Master's degree in requirements ✓")
            for col in ['company_name', 'requirements_and_qualifications', 'responsibilities', 'salary', 'experience_years']:
                not_specified = (df[col] == 'Not specified').sum()
                percentage = (not_specified / len(df)) * 100
                print(f"  {col}: {len(df) - not_specified}/{len(df)} complete ({percentage:.1f}% missing)")

            print(f"{'='*60}\n")
            return df
        else:
            print("No data to save.")
            return None

    def close(self):
        """Close the browser"""
        self.driver.quit()


def main(jobs_per_designation=None, designations=None):
    """Main execution function

    Args:
        jobs_per_designation: Number of jobs to scrape per designation.
                             If None, will prompt user for input.
        designations: List of job designations to scrape.
                     If None, will prompt user for input.
    """
    # Get job designations
    if designations is None:
        print("\n" + "="*60)
        print("INDEED JOB SCRAPER")
        print("="*60)
        print("\nDefault designations:")
        default_designations = [
            'Business Analyst',
            'Data Analyst',
            'Data Scientist'
        ]
        for i, des in enumerate(default_designations, 1):
            print(f"  {i}. {des}")

        print("\n" + "-"*60)
        use_default = input("\nUse default designations? (yes/no): ").lower()

        if use_default in ['yes', 'y']:
            designations = default_designations
        else:
            designations = []
            print("\n" + "-"*60)
            print("Enter job designations to scrape (one per line)")
            print("Note: The scraper will automatically capture all levels")
            print("(Junior, Senior, I, II, III, Lead, Principal, etc.)")
            print("Type 'done' when finished")
            print("-"*60 + "\n")

            while True:
                designation = input(f"Enter designation #{len(designations) + 1} (or 'done'): ").strip()

                if designation.lower() == 'done':
                    if len(designations) == 0:
                        print("⚠️  You must enter at least one designation.")
                        continue
                    break

                if designation:
                    designations.append(designation)
                    print(f"✓ Added: {designation}")
                else:
                    print("⚠️  Please enter a valid designation.")

        print("\n" + "="*60)
        print("Designations to scrape:")
        for i, des in enumerate(designations, 1):
            print(f"  {i}. {des}")
        print("="*60)
    else:
        print("\n" + "="*60)
        print("INDEED JOB SCRAPER")
        print("="*60)
        print(f"\nDesignations to scrape:")
        for i, des in enumerate(designations, 1):
            print(f"  {i}. {des}")
        print("="*60)

    # Get number of jobs per designation
    if jobs_per_designation is None:
        print("\n")

        while True:
            try:
                jobs_input = input("Enter number of jobs to scrape per designation (e.g., 10, 50, 500): ")
                jobs_per_designation = int(jobs_input)

                if jobs_per_designation <= 0:
                    print("Please enter a positive number.")
                    continue
                elif jobs_per_designation > 1000:
                    confirm = input(f"⚠️  You want to scrape {jobs_per_designation} jobs per designation ({jobs_per_designation * len(designations)} total). This may take a long time. Continue? (yes/no): ")
                    if confirm.lower() not in ['yes', 'y']:
                        continue

                break
            except ValueError:
                print("Invalid input. Please enter a number.")

        print(f"\n{'='*60}")
        print(f"Starting scrape: {jobs_per_designation} jobs per designation")
        print(f"Total jobs to collect: {jobs_per_designation * len(designations)}")
        print(f"{'='*60}\n")
    else:
        print("\n" + "="*60)
        print(f"Scraping {jobs_per_designation} jobs per designation")
        print(f"{'='*60}\n")

    # Initialize scraper
    scraper = IndeedJobScraper(headless=False)  # Set to True for headless mode

    try:
        # Scrape jobs across all designations simultaneously
        total_collected = scraper.search_jobs(designations, jobs_per_designation)

        # Save all data to CSV
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        filename = f'indeed_jobs_{jobs_per_designation}per_{timestamp}.csv'
        df = scraper.save_to_csv(filename)

        # Display summary
        if df is not None:
            print("\n" + "="*60)
            print("SCRAPING SUMMARY")
            print("="*60)
            print(f"\nJobs by Search Category:")
            print(df['search_category'].value_counts())
            print(f"\nSample Job Titles Found:")
            print(df['job_title'].value_counts().head(10))
            print(f"\nSample Data:")
            print(df.head(3)[['search_category', 'job_title', 'company_name', 'salary', 'experience_years']])

    finally:
        # Close browser
        scraper.close()


if __name__ == "__main__":
    # Option 1: Interactive mode - will prompt for designations and job count
    main()

    # Option 2: Specify designations directly
    # custom_designations = ['Financial Analyst', 'Machine Learning Engineer', 'Product Manager']
    # main(jobs_per_designation=50, designations=custom_designations)

    # Option 3: Use defaults but specify job count
    # main(jobs_per_designation=10)  # Uses default designations with 10 jobs each


INDEED JOB SCRAPER

Default designations:
  1. Business Analyst
  2. Data Analyst
  3. Data Scientist

------------------------------------------------------------



Use default designations? (yes/no):  yes



Designations to scrape:
  1. Business Analyst
  2. Data Analyst
  3. Data Scientist




Enter number of jobs to scrape per designation (e.g., 10, 50, 500):  500



Starting scrape: 500 jobs per designation
Total jobs to collect: 1500


Starting multi-category scraping...
Target: 500 jobs per designation


--- Searching for Business Analyst (Need 500 more) ---
  Error extracting description: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="jobDescriptionText"]"}
  (Session info: chrome=142.0.7444.176); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
0   chromedriver                        0x000000010141aecc cxxbridge1$str$ptr + 2941512
1   chromedriver                        0x0000000101412b88 cxxbridge1$str$ptr + 2907908
2   chromedriver                        0x0000000100f2a2b0 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74020
3   chromedriver                        0x0000000100f7188c _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 366336
4   chromedriv

KeyboardInterrupt: 

Skills extraction


In [ ]:
# step3_skills_only.py
# Inputs:
#   /mnt/data/MSBA-top5.csv            (curricula; must have a university-like column)
#   /mnt/data/cleaned_job_data.csv     (jobs; should have role/title and job description)
# Outputs:
#   /mnt/data/skills_from_curriculum.csv   -> columns: university, skill, count
#   /mnt/data/skills_from_jobs.csv         -> columns: role, skill, count
#
# No displays, no cosine, no fluff.

import re
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

CURR = "/Users/varsha/Desktop/Fall sem/Unstructured data/Project/MSBA_top_10.csv"
JOBS = "cleaned_job_data.csv"
OUT  = Path('/Users/varsha/Desktop/Fall sem/Unstructured data/Project')
OUT.mkdir(parents=True, exist_ok=True)

# ---------------- helpers ----------------
def pick_cols(df, hints):
    return [c for c in df.columns if any(h in c.lower() for h in hints)]

def collapse_text(df, cols):
    if not cols:
        return pd.Series([""] * len(df))
    block = df[cols].astype(str).fillna("")
    return block.apply(lambda r: " ".join([x for x in r if x.strip()]), axis=1)

def norm_variant(s: str) -> str:
    s = s.lower().strip()
    s = re.sub(r"[-_/]", " ", s)
    s = re.sub(r"\s+", " ", s)
    return s

# ---------------- expanded skill vocab ----------------
RAW_SKILLS = [
    # Languages & query
    "python","r","java","c","c++","c#","scala","julia","matlab","sql","tsql","nosql",
    # BI & viz
    "tableau","power bi","looker","qlik","excel","vba","superset","metabase",
    # Py data & viz
    "pandas","numpy","scipy","statsmodels","matplotlib","plotly","bokeh","seaborn",
    # ML/DL libs
    "scikit-learn","sklearn","xgboost","lightgbm","catboost","tensorflow","pytorch","keras","fastai","prophet","gluonts",
    # NLP/CV
    "spacy","nltk","transformers","huggingface","gensim","word2vec","bert","roberta","gpt","opencv",
    # Big data
    "spark","pyspark","hadoop","mapreduce","hive","pig","hbase","kafka","flink","dask","ray",
    # Orchestration & MLOps
    "airflow","luigi","mlflow","prefect","dagster",
    # Warehouses & db
    "snowflake","bigquery","redshift","databricks","athena","glue","synapse","dbt",
    "mysql","postgresql","postgres","mssql","oracle","sqlite","duckdb","clickhouse","elasticsearch","neo4j","mongodb","cassandra","dynamodb",
    # Cloud
    "aws","azure","gcp","s3","ec2","emr","lambda","dataproc",
    # DevOps/containers
    "git","github","gitlab","docker","kubernetes","terraform",
    # Business systems
    "sap","salesforce","netsuite",
    # Methods / themes
    "machine learning","deep learning","nlp","natural language processing","computer vision",
    "statistics","statistical inference","hypothesis testing","a/b testing","experimentation",
    "time series","forecasting","bayesian","mcmc",
    "optimization","linear programming","mixed integer programming","mip",
    "causal inference","propensity score","matching",
    "feature engineering","model interpretability","shap","lime","calibration","cross validation",
    "dimensionality reduction","pca","cluster analysis","k means","k-means",
    "data visualization","data viz","data mining","data warehousing","data warehouse",
    "data modeling","dimensional modeling","star schema","snowflake schema",
    "data engineering","data governance","data quality","data ethics",
    "cloud computing","cloud analytics","business analytics","marketing analytics",
    "text analytics","predictive analytics","prescriptive analytics",
    "big data","database systems","information systems",
    "mlops","model deployment","model monitoring",
]
ALIASES = {
    "scikit-learn": ["scikit learn","sklearn"],
    "power bi": ["powerbi","power-bi"],
    "postgresql": ["postgres"],
    "a/b testing": ["ab testing","a b testing","a-b testing"],
    "k-means": ["k means"],
    "nlp": ["natural language processing"],
    "mip": ["mixed integer programming"],
    "data viz": ["data visualization"],
    "data warehouse": ["data warehousing"],
    "opencv": ["open cv"],
    "xgboost": ["xgb"],
    "mlops": ["ml ops"],
}

def build_skill_vectorizer():
    # Build variant -> canonical and vocabulary dict
    variant_to_canon = {}
    variants = []
    for canon in RAW_SKILLS:
        for v in [canon] + ALIASES.get(canon, []):
            nv = norm_variant(v)
            variants.append(nv)
            # Use canonical normalized form as the target
            variant_to_canon[nv] = norm_variant(canon)
    # dedupe while preserving order
    seen, vocab_list = set(), []
    for v in variants:
        if v not in seen and v.strip():
            seen.add(v); vocab_list.append(v)
    vocab_dict = {term: i for i, term in enumerate(vocab_list)}
    vec = CountVectorizer(
        vocabulary=vocab_dict,
        ngram_range=(1,4),  # catch multiword skills and variants
        token_pattern=r"[A-Za-z0-9\+\#]+"
    )
    idx_to_variant = {i: term for term, i in vocab_dict.items()}
    return vec, idx_to_variant, variant_to_canon

def skills_from_row(row_vector, idx_to_variant, variant_to_canon):
    idxs = row_vector.indices
    return sorted({ variant_to_canon[idx_to_variant[i]] for i in idxs })

# ---------------- load data ----------------
curr = pd.read_csv(CURR)
jobs = pd.read_csv(JOBS)

# curriculum text + university
cur_text_cols = pick_cols(curr, ["curriculum","course","syllabus","program","msba","catalog","description"]) \
                or [c for c in curr.columns if curr[c].dtype == object][:1]
curr["curriculum_text"] = collapse_text(curr, cur_text_cols)

uni_cands = pick_cols(curr, ["university","school","college","institution"])
if not uni_cands:
    raise ValueError("No university column found in MSBA-top5.csv")
UNI_COL = uni_cands[0]
curr["university"] = (
    curr[UNI_COL].astype(str).str.strip().str.replace(r"\s+", " ", regex=True).str.title()
)

# job text + role
job_text_cols = [c for c in jobs.columns if c.lower() in {"final_job_description","job_description","description"}] \
                or pick_cols(jobs, ["job","description","posting","jd"]) \
                or [c for c in jobs.columns if jobs[c].dtype == object][:1]
jobs["job_text"] = collapse_text(jobs, job_text_cols)

role_cands = pick_cols(jobs, ["role","title","job_title","position"])
ROLE_COL = role_cands[0] if role_cands else "Role"
if ROLE_COL not in jobs.columns:
    jobs[ROLE_COL] = "Unknown Role"

# ---------------- extract skills ----------------
skill_vec, idx_to_variant, variant_to_canon = build_skill_vectorizer()

X_cur = skill_vec.transform(curr["curriculum_text"].fillna("").tolist())
X_job = skill_vec.transform(jobs["job_text"].fillna("").tolist())

curr["doc_skills"] = [skills_from_row(X_cur.getrow(i), idx_to_variant, variant_to_canon)
                      for i in range(X_cur.shape[0])]
jobs["doc_skills"] = [skills_from_row(X_job.getrow(i), idx_to_variant, variant_to_canon)
                      for i in range(X_job.shape[0])]

# ---------------- aggregate to exactly two outputs ----------------
skills_from_curriculum = (
    curr[["university","doc_skills"]]
      .explode("doc_skills")
      .dropna(subset=["doc_skills"])
      .groupby(["university","doc_skills"])
      .size().reset_index(name="count")
      .rename(columns={"doc_skills":"skill"})
      .sort_values(["university","count"], ascending=[True,False])
)

skills_from_jobs = (
    jobs[[ROLE_COL,"doc_skills"]]
      .explode("doc_skills")
      .dropna(subset=["doc_skills"])
      .groupby([ROLE_COL,"doc_skills"])
      .size().reset_index(name="count")
      .rename(columns={ROLE_COL:"role","doc_skills":"skill"})
      .sort_values(["role","count"], ascending=[True,False])
)

skills_from_curriculum.to_csv(OUT/"skills_from_curriculum.csv", index=False)
skills_from_jobs.to_csv(OUT/"skills_from_jobs.csv", index=False)

print("Saved:")
print(OUT/"skills_from_curriculum.csv")
print(OUT/"skills_from_jobs.csv")


In [ ]:
#!/usr/bin/env python3
"""
Top-5 skills-only n-grams per role (first column) + optional PPT slide.

Inputs:
  cleaned_job_data.csv  (first column = role bucket; a later column = job description)

Outputs:
  job_top5_skill_ngrams_by_first_column_roles.csv
  Top_Skill_Signals_by_Role.pptx  (if MAKE_PPT=True)

Edit the PATHS section if your file lives somewhere else.
"""

import re
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# ============ PATHS & SETTINGS ============
INPUT_CSV = "cleaned_job_data.csv"   # change if needed
OUT_DIR   = Path(".")                # change if needed
TOP_K     = 5                        # top n-grams per role
MAKE_PPT  = True                     # set False if you don't want the PPT
# =========================================

OUT_DIR.mkdir(parents=True, exist_ok=True)

def norm(s: str) -> str:
    s = s.lower().strip()
    s = re.sub(r"[-_/]", " ", s)
    s = re.sub(r"\s+", " ", s)
    return s

def collapse_text(series: pd.Series) -> pd.Series:
    s = series.astype(str).fillna("")
    return s.apply(lambda x: re.sub(r"\s+", " ", x.strip()))

# A practical skills dictionary: tools, platforms, and methods
SKILL_TERMS = [
    # Languages & DB/query
    "python","r","java","c","c++","c#","scala","julia","matlab","sql","tsql","nosql",
    "mysql","postgresql","postgres","mssql","oracle","sqlite","duckdb","clickhouse",
    # BI & viz
    "tableau","power bi","powerbi","looker","qlik","excel","vba","metabase","superset",
    "data visualization","dashboard","dashboards",
    # Py data & viz libs
    "pandas","numpy","scipy","statsmodels","matplotlib","plotly","bokeh","seaborn",
    # ML/DL libs & models
    "scikit-learn","sklearn","xgboost","lightgbm","catboost","tensorflow","pytorch","keras","fastai",
    # NLP/CV
    "spacy","nltk","transformers","huggingface","gensim","word2vec","bert","roberta","gpt","opencv",
    # Data eng / big data
    "spark","pyspark","hadoop","mapreduce","hive","kafka","flink","dask","ray","airflow","luigi","dbt",
    # Warehouses & cloud
    "snowflake","bigquery","redshift","databricks","athena","glue","synapse",
    "aws","azure","gcp","s3","ec2","emr","lambda","dataproc",
    # DevOps / MLOps
    "git","github","gitlab","docker","kubernetes","terraform","mlflow","prefect","dagster",
    # Methods / concepts
    "machine learning","deep learning","nlp","natural language processing","computer vision",
    "statistics","hypothesis testing","a/b testing","ab testing","experimentation",
    "causal inference","propensity score","matching",
    "feature engineering","model interpretability","shap","lime","calibration","cross validation",
    "time series","forecasting","arima","sarima","bayesian","mcmc",
    "optimization","linear programming","mixed integer programming","mip",
    "regression","classification","clustering","pca","dimensionality reduction",
    # Data mgmt themes
    "etl","elt","data modeling","dimensional modeling","star schema","snowflake schema",
    "data warehousing","data warehouse","data governance","data quality","data engineering",
    "cloud analytics","business analytics","text analytics","predictive analytics","prescriptive analytics",
]

ALIASES = {
    "scikit-learn": ["scikit learn","sklearn"],
    "power bi": ["power-bi","powerbi"],
    "a/b testing": ["ab testing","a b testing","a-b testing"],
    "postgresql": ["postgres"],
}

def build_vocab_dict():
    variants = []
    for canon in SKILL_TERMS:
        for v in [canon] + ALIASES.get(canon, []):
            v = norm(v)
            variants.append(v)
            # add simple plural/ing variants for the last token
            toks = v.split()
            if toks:
                last = toks[-1]
                if not last.endswith("s"):
                    variants.append(norm(" ".join(toks[:-1] + [last + "s"])))
                if last.endswith("e"):
                    variants.append(norm(" ".join(toks[:-1] + [last[:-1] + "ing"])))
                elif not last.endswith("ing") and last.isalpha():
                    variants.append(norm(" ".join(toks[:-1] + [last + "ing"])))
    # dedupe preserving order
    seen, vocab = set(), []
    for t in variants:
        if t and t not in seen:
            seen.add(t); vocab.append(t)
    return {term: i for i, term in enumerate(vocab)}

def detect_text_col(df: pd.DataFrame, exclude_cols):
    # try standard columns first
    candidates = [c for c in df.columns if c.lower() in {"final_job_description","job_description","description"} and c not in exclude_cols]
    if candidates:
        return candidates[0]
    # fallback to first object-like column not excluded
    for c in df.columns:
        if c in exclude_cols:
            continue
        if df[c].dtype == object:
            return c
    # absolute last resort: next column
    for c in df.columns:
        if c not in exclude_cols:
            return c

def compute_topk_by_role(input_csv: str, top_k: int = 5) -> pd.DataFrame:
    df = pd.read_csv(input_csv)
    if df.shape[1] < 2:
        raise ValueError("Need at least two columns: first = role bucket, plus a text column.")

    role_col = df.columns[0]  # first column as requested
    text_col = detect_text_col(df, exclude_cols={role_col})

    df["job_text"] = collapse_text(df[text_col])

    vocab_dict = build_vocab_dict()
    vec = CountVectorizer(
        vocabulary=vocab_dict,
        ngram_range=(1,4),
        token_pattern=r"[A-Za-z0-9\\+\\#]+"
    )
    X = vec.transform(df["job_text"].tolist())

    rows = []
    # group by first column (role bucket)
    for role, idx in df.groupby(role_col).indices.items():
        sub = X[idx, :]
        freqs = np.asarray(sub.sum(axis=0)).ravel()
        terms = np.array(sorted(vocab_dict, key=lambda k: vocab_dict[k]))
        order = np.argsort(-freqs)
        top = [(terms[i], int(freqs[i])) for i in order if freqs[i] > 0][:top_k]
        for rank, (ng, fr) in enumerate(top, start=1):
            rows.append({"role": str(role), "ngram": ng, "frequency": fr, "rank": rank})

    out_df = pd.DataFrame(rows, columns=["role","ngram","frequency","rank"]).sort_values(["role","rank"])
    return out_df

def save_ppt_from_table(df: pd.DataFrame, out_path: Path):
    try:
        # lazy import so people without python-pptx can still run CSV part
        from pptx import Presentation
        from pptx.util import Inches, Pt
        from pptx.enum.shapes import MSO_SHAPE
        from pptx.dml.color import RGBColor
        from pptx.enum.text import PP_ALIGN
    except Exception as e:
        print("python-pptx not installed; skipping PPT. pip install python-pptx if you want it.")
        return

    prs = Presentation()
    slide = prs.slides.add_slide(prs.slide_layouts[6])

    # Title
    title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.3), Inches(9.0), Inches(0.7))
    tf = title_box.text_frame
    tf.text = "Top Skill Signals by Role (Jobs)"
    tf.paragraphs[0].font.size = Pt(28)
    tf.paragraphs[0].font.bold = True

    colors = {
        "data analyst": RGBColor(0x25, 0x63, 0xEB),
        "business analyst": RGBColor(0x08, 0x91, 0xB2),
        "data scientist": RGBColor(0x16, 0xA3, 0x4A),
        "data engineer": RGBColor(0x93, 0x33, 0xEA),
        "ml engineer": RGBColor(0xDC, 0x26, 0x26),
        "other": RGBColor(0x64, 0x74, 0x8B),
    }

    def add_card(left, top, role_name, rows5):
        card_w, card_h = Inches(4.0), Inches(2.1)
        card = slide.shapes.add_shape(MSO_SHAPE.ROUNDED_RECTANGLE, left, top, card_w, card_h)
        card.fill.solid(); card.fill.fore_color.rgb = RGBColor(0xFF,0xFF,0xFF)
        card.line.color.rgb = RGBColor(0xEE,0xEE,0xEE)

        pill = slide.shapes.add_shape(MSO_SHAPE.ROUNDED_RECTANGLE, left+Inches(0.25), top+Inches(0.2), card_w-Inches(0.5), Inches(0.4))
        pill.fill.solid(); pill.fill.fore_color.rgb = colors.get(role_name.lower(), RGBColor(0x44,0x44,0x44))
        pill.line.fill.background()
        pt = pill.text_frame; pt.clear()
        p = pt.paragraphs[0]; p.text = role_name.upper(); p.font.color.rgb = RGBColor(0xFF,0xFF,0xFF); p.font.size = Pt(12); p.font.bold = True; p.alignment = PP_ALIGN.CENTER

        tb = slide.shapes.add_textbox(left+Inches(0.3), top+Inches(0.75), card_w-Inches(0.6), card_h-Inches(0.95))
        ttf = tb.text_frame; ttf.word_wrap = True
        for i, (skill, freq) in enumerate(rows5):
            para = ttf.add_paragraph() if i else ttf.paragraphs[0]
            para.text = f"{skill} — {freq}"
            para.font.size = Pt(14)
            if i == 0: para.font.bold = True

    # choose three most common roles by row count or just the distinct found
    roles = list(df["role"].unique())
    # layout: three cards across
    lefts = [Inches(0.5), Inches(4.8), Inches(9.1)]
    top = Inches(1.2)

    for idx, role in enumerate(roles[:3]):  # first three roles in the file
        top5 = df[df["role"] == role].sort_values("rank")[["ngram","frequency"]].values.tolist()
        add_card(lefts[idx], top, role, top5)

    prs.save(str(out_path))

if __name__ == "__main__":
    out_df = compute_topk_by_role(INPUT_CSV, TOP_K)
    csv_path = OUT_DIR / "job_top5_skill_ngrams_by_first_column_roles.csv"
    out_df.to_csv(csv_path, index=False)
    print(f"Saved CSV: {csv_path.resolve()}")

    if MAKE_PPT:
        ppt_path = OUT_DIR / "Top_Skill_Signals_by_Role.pptx"
        save_ppt_from_table(out_df, ppt_path)
        print(f"Saved PPT: {ppt_path.resolve()}")


# **TOP 5 US**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for better-looking plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Read the CSV files
curriculum_df = pd.read_csv('skills_from_curriculum.csv')
job_skills_df = pd.read_csv('step3_job_skills_by_role.csv')

# Function to normalize skill names for better matching
def normalize_skill(skill):
    """Normalize skill names by converting to lowercase and removing special characters"""
    skill = str(skill).lower()
    skill = re.sub(r'[^a-z0-9]', ' ', skill)
    skill = re.sub(r'\s+', ' ', skill)
    return skill.strip()

# Normalize skills in both dataframes
curriculum_df['skill_normalized'] = curriculum_df['skill'].apply(normalize_skill)
job_skills_df['skill_normalized'] = job_skills_df['job_skills'].apply(normalize_skill)

# Define role categories with keyword matching patterns
# Keywords are ordered by priority (more specific first)
role_categories = {
    'Data Scientist': ['data scientist', 'ds ', 'data science'],
    'Data Analyst': ['data analyst', 'data analysis'],
    'Business Analyst': ['business analyst', 'business intelligence analyst', 'bi analyst', 'business systems analyst']
}

# Create a category column for job roles using keyword matching
def categorize_role(role):
    """
    Categorize job role based on keyword matching.
    Uses priority-based matching to handle overlapping keywords.
    """
    role_lower = role.lower()

    # Check each category's keywords
    for category, keywords in role_categories.items():
        for keyword in keywords:
            if keyword in role_lower:
                return category
    return None

job_skills_df['role_category'] = job_skills_df['role'].apply(categorize_role)

# Print categorization statistics
print("\n" + "="*80)
print("ROLE CATEGORIZATION SUMMARY")
print("="*80 + "\n")

for category in role_categories.keys():
    count = len(job_skills_df[job_skills_df['role_category'] == category])
    unique_roles = job_skills_df[job_skills_df['role_category'] == category]['role'].nunique()
    print(f"{category}:")
    print(f"  • Total records: {count}")
    print(f"  • Unique role titles: {unique_roles}")

    # Show sample role titles
    sample_roles = job_skills_df[job_skills_df['role_category'] == category]['role'].unique()[:5]
    print(f"  • Sample roles: {', '.join(sample_roles)}")
    print()

uncategorized = len(job_skills_df[job_skills_df['role_category'].isna()])
print(f"Uncategorized roles: {uncategorized}")
print(f"Total categorized: {len(job_skills_df[job_skills_df['role_category'].notna()])}")
print()

# Plot 1: Role Categorization Distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart for categorization distribution
category_counts = job_skills_df[job_skills_df['role_category'].notna()].groupby('role_category').size()
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
ax1.pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%',
        startangle=90, colors=colors, textprops={'fontsize': 11, 'weight': 'bold'})
ax1.set_title('Distribution of Job Roles by Category', fontsize=14, weight='bold', pad=20)

# Bar chart for unique role titles per category
unique_counts = []
for category in role_categories.keys():
    count = job_skills_df[job_skills_df['role_category'] == category]['role'].nunique()
    unique_counts.append(count)

ax2.bar(role_categories.keys(), unique_counts, color=colors, edgecolor='black', linewidth=1.5)
ax2.set_title('Unique Job Titles per Category', fontsize=14, weight='bold', pad=20)
ax2.set_ylabel('Number of Unique Titles', fontsize=11, weight='bold')
ax2.set_xlabel('Role Category', fontsize=11, weight='bold')
ax2.tick_params(axis='x', rotation=45)
for i, v in enumerate(unique_counts):
    ax2.text(i, v + 1, str(v), ha='center', va='bottom', fontsize=10, weight='bold')

plt.tight_layout()
plt.show()
print("📊 Plot displayed: Role Categorization Distribution\n")

# Filter to only keep categorized roles
job_skills_df = job_skills_df[job_skills_df['role_category'].notna()]

# Get all unique skills across both datasets
all_skills = sorted(set(
    list(curriculum_df['skill_normalized'].unique()) +
    list(job_skills_df['skill_normalized'].unique())
))

# Create university skill vectors
universities = curriculum_df['university'].unique()
university_vectors = {}

for uni in universities:
    uni_skills = curriculum_df[curriculum_df['university'] == uni]
    skill_vector = {}
    for skill in all_skills:
        count = uni_skills[uni_skills['skill_normalized'] == skill]['count'].sum()
        skill_vector[skill] = count
    university_vectors[uni] = skill_vector

# Create job role skill vectors (aggregated by category)
role_vectors = {}

for category in role_categories.keys():
    category_skills = job_skills_df[job_skills_df['role_category'] == category]
    skill_vector = {}
    for skill in all_skills:
        count = category_skills[category_skills['skill_normalized'] == skill]['count'].sum()
        skill_vector[skill] = count
    role_vectors[category] = skill_vector

# Convert dictionaries to matrices for cosine similarity calculation
def dict_to_vector(skill_dict, all_skills):
    """Convert skill dictionary to numpy array"""
    return np.array([skill_dict.get(skill, 0) for skill in all_skills])

# Calculate cosine similarity for each university-role combination
results = []

for role in role_categories.keys():
    role_vector = dict_to_vector(role_vectors[role], all_skills)

    print(f"\n{'='*80}")
    print(f"ALIGNMENT ANALYSIS FOR: {role}")
    print(f"{'='*80}\n")

    role_similarities = []

    for uni in universities:
        uni_vector = dict_to_vector(university_vectors[uni], all_skills)

        # Calculate cosine similarity
        similarity = cosine_similarity(
            uni_vector.reshape(1, -1),
            role_vector.reshape(1, -1)
        )[0][0]

        # Find matching skills (both have non-zero values)
        matching_skills = [
            skill for skill in all_skills
            if university_vectors[uni][skill] > 0 and role_vectors[role][skill] > 0
        ]

        # Get top matching skills by job demand
        top_skills = sorted(
            matching_skills,
            key=lambda s: role_vectors[role][s],
            reverse=True
        )[:5]

        role_similarities.append({
            'University': uni,
            'Role': role,
            'Similarity_Score': similarity * 100,
            'Matching_Skills_Count': len(matching_skills),
            'Top_Matching_Skills': ', '.join(top_skills) if top_skills else 'None'
        })

    # Sort by similarity score
    role_similarities = sorted(
        role_similarities,
        key=lambda x: x['Similarity_Score'],
        reverse=True
    )

    # Print rankings
    for rank, item in enumerate(role_similarities, 1):
        medal = '🥇' if rank == 1 else '🥈' if rank == 2 else '🥉' if rank == 3 else '  '
        print(f"{medal} Rank {rank}: {item['University']}")
        print(f"   Similarity Score: {item['Similarity_Score']:.2f}%")
        print(f"   Matching Skills: {item['Matching_Skills_Count']}")
        print(f"   Top Skills: {item['Top_Matching_Skills']}")
        print()

    results.extend(role_similarities)

# Create comprehensive results dataframe
results_df = pd.DataFrame(results)

# Save to CSV
results_df.to_csv('university_job_alignment_results.csv', index=False)
print(f"\nResults saved to 'university_job_alignment_results.csv'")

# Create summary pivot table
print(f"\n{'='*80}")
print("SUMMARY: Alignment Scores by University and Role")
print(f"{'='*80}\n")

pivot_table = results_df.pivot_table(
    values='Similarity_Score',
    index='University',
    columns='Role',
    aggfunc='first'
).round(2)

print(pivot_table)

# Plot 2: Heatmap of Similarity Scores
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, fmt='.2f', cmap='YlOrRd',
            cbar_kws={'label': 'Similarity Score (%)'},
            linewidths=2, linecolor='white',
            square=True, vmin=0, vmax=100)
plt.title('University-Role Alignment Heatmap\n(Cosine Similarity Scores)',
          fontsize=14, weight='bold', pad=20)
plt.xlabel('Role Category', fontsize=11, weight='bold')
plt.ylabel('University', fontsize=11, weight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()
print("📊 Plot displayed: Similarity Heatmap\n")

# Find best university for each role
print(f"\n{'='*80}")
print("BEST UNIVERSITY FOR EACH ROLE")
print(f"{'='*80}\n")

for role in role_categories.keys():
    best = results_df[results_df['Role'] == role].sort_values(
        'Similarity_Score',
        ascending=False
    ).iloc[0]
    print(f"{role}:")
    print(f"  🏆 {best['University']}")
    print(f"  Score: {best['Similarity_Score']:.2f}%")
    print(f"  Matching Skills: {best['Matching_Skills_Count']}")
    print()

# Plot 3: Grouped Bar Chart - Similarity Scores by Role
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(universities))
width = 0.25

for i, role in enumerate(role_categories.keys()):
    scores = [results_df[(results_df['University'] == uni) &
                         (results_df['Role'] == role)]['Similarity_Score'].values[0]
              for uni in universities]
    ax.bar(x + i*width, scores, width, label=role, edgecolor='black', linewidth=1)

ax.set_xlabel('University', fontsize=11, weight='bold')
ax.set_ylabel('Similarity Score (%)', fontsize=11, weight='bold')
ax.set_title('University-Role Alignment Comparison', fontsize=14, weight='bold', pad=20)
ax.set_xticks(x + width)
ax.set_xticklabels([uni.split('(')[0].strip() for uni in universities], rotation=45, ha='right')
ax.legend(title='Role Category', fontsize=10, title_fontsize=11)
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()
print("📊 Plot displayed: Similarity Comparison Bars\n")

# Plot 4: Best University for Each Role (Visual Summary)
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
colors_role = ['#FF6B6B', '#4ECDC4', '#45B7D1']

for idx, (role, color) in enumerate(zip(role_categories.keys(), colors_role)):
    role_data = results_df[results_df['Role'] == role].sort_values('Similarity_Score', ascending=True)

    axes[idx].barh(range(len(role_data)), role_data['Similarity_Score'],
                   color=color, edgecolor='black', linewidth=1.5)
    axes[idx].set_yticks(range(len(role_data)))
    axes[idx].set_yticklabels([uni.split('(')[0].strip() for uni in role_data['University']], fontsize=9)
    axes[idx].set_xlabel('Similarity Score (%)', fontsize=10, weight='bold')
    axes[idx].set_title(f'{role}', fontsize=12, weight='bold', pad=10)
    axes[idx].grid(axis='x', alpha=0.3)

    # Add value labels
    for i, v in enumerate(role_data['Similarity_Score']):
        axes[idx].text(v + 0.5, i, f'{v:.1f}%', va='center', fontsize=9, weight='bold')

plt.suptitle('University Rankings by Role Category', fontsize=16, weight='bold', y=1.02)
plt.tight_layout()
plt.show()
print("📊 Plot displayed: Rankings by Role\n")

# Create a detailed skills comparison
print(f"\n{'='*80}")
print("DETAILED SKILLS ANALYSIS")
print(f"{'='*80}\n")

for role in role_categories.keys():
    print(f"\n{role} - Top 10 Required Skills:")
    role_skills = sorted(
        [(skill, role_vectors[role][skill]) for skill in all_skills if role_vectors[role][skill] > 0],
        key=lambda x: x[1],
        reverse=True
    )[:10]

    for skill, count in role_skills:
        print(f"  • {skill}: {count} job postings")

    # Show which universities teach these top skills
    print(f"\n  Universities teaching these skills:")
    for uni in universities:
        taught_count = sum(1 for skill, _ in role_skills if university_vectors[uni][skill] > 0)
        coverage = (taught_count / len(role_skills)) * 100
        print(f"    {uni}: {taught_count}/{len(role_skills)} skills ({coverage:.1f}% coverage)")
    print()

# Plot 5: Skills Coverage Analysis
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, role in enumerate(role_categories.keys()):
    # Get top 10 skills for this role
    role_skills = sorted(
        [(skill, role_vectors[role][skill]) for skill in all_skills if role_vectors[role][skill] > 0],
        key=lambda x: x[1],
        reverse=True
    )[:10]

    # Calculate coverage for each university
    coverage_data = []
    for uni in universities:
        taught_count = sum(1 for skill, _ in role_skills if university_vectors[uni][skill] > 0)
        coverage_pct = (taught_count / len(role_skills)) * 100
        coverage_data.append(coverage_pct)

    axes[idx].bar(range(len(universities)), coverage_data,
                  color=colors_role[idx], edgecolor='black', linewidth=1.5)
    axes[idx].set_xticks(range(len(universities)))
    axes[idx].set_xticklabels([uni.split('(')[0].strip() for uni in universities],
                               rotation=45, ha='right', fontsize=9)
    axes[idx].set_ylabel('Coverage (%)', fontsize=10, weight='bold')
    axes[idx].set_title(f'{role}\nTop 10 Skills Coverage', fontsize=11, weight='bold')
    axes[idx].set_ylim(0, 100)
    axes[idx].grid(axis='y', alpha=0.3)

    # Add value labels
    for i, v in enumerate(coverage_data):
        axes[idx].text(i, v + 2, f'{v:.0f}%', ha='center', va='bottom', fontsize=9, weight='bold')

plt.suptitle('University Coverage of Top 10 In-Demand Skills by Role',
             fontsize=14, weight='bold', y=1.02)
plt.tight_layout()
plt.show()
print("📊 Plot displayed: Skills Coverage Analysis\n")

# Plot 6: Matching Skills Count Comparison
fig, ax = plt.subplots(figsize=(12, 6))
matching_pivot = results_df.pivot_table(
    values='Matching_Skills_Count',
    index='University',
    columns='Role',
    aggfunc='first'
)

matching_pivot.plot(kind='bar', ax=ax, edgecolor='black', linewidth=1.5)
ax.set_xlabel('University', fontsize=11, weight='bold')
ax.set_ylabel('Number of Matching Skills', fontsize=11, weight='bold')
ax.set_title('Number of Matching Skills Between Curriculum and Job Requirements',
             fontsize=14, weight='bold', pad=20)
ax.set_xticklabels([uni.split('(')[0].strip() for uni in matching_pivot.index],
                    rotation=45, ha='right')
ax.legend(title='Role Category', fontsize=10, title_fontsize=11)
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()
print("📊 Plot displayed: Matching Skills Comparison\n")

# Plot 7: Radar Chart for Overall Alignment
from math import pi

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

angles = [n / float(len(role_categories)) * 2 * pi for n in range(len(role_categories))]
angles += angles[:1]

for uni in universities:
    values = []
    for role in role_categories.keys():
        score = results_df[(results_df['University'] == uni) &
                          (results_df['Role'] == role)]['Similarity_Score'].values[0]
        values.append(score)
    values += values[:1]

    ax.plot(angles, values, 'o-', linewidth=2, label=uni.split('(')[0].strip())
    ax.fill(angles, values, alpha=0.15)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(role_categories.keys(), fontsize=11, weight='bold')
ax.set_ylim(0, 100)
ax.set_ylabel('Similarity Score (%)', fontsize=11, weight='bold', labelpad=30)
ax.set_title('Overall University-Role Alignment Profile',
             fontsize=14, weight='bold', pad=30)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=10)
ax.grid(True)
plt.tight_layout()
plt.show()
print("📊 Plot displayed: Alignment Radar Chart\n")

print("\n" + "="*80)
print("ALL VISUALIZATIONS DISPLAYED!")
print("="*80)
print("\n📊 Displayed plots:")
print("  1. Role Categorization Distribution - Role distribution and unique titles")
print("  2. Similarity Heatmap - Heatmap of all similarity scores")
print("  3. Similarity Comparison Bars - Grouped bar chart comparison")
print("  4. Rankings by Role - Horizontal bar charts showing rankings")
print("  5. Skills Coverage Analysis - Top 10 skills coverage by university")
print("  6. Matching Skills Comparison - Number of matching skills")
print("  7. Alignment Radar Chart - Overall alignment profile radar chart")
print("\n")

In [ ]:
# --- Curriculum vs Job Skill Alignment for All Roles ---

import pandas as pd
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# --- Load data ---
curriculum_df = pd.read_csv('skills_from_curriculum.csv')
job_skills_df = pd.read_csv('step3_job_skills_by_role.csv')

# --- 1️⃣ Normalize skill names ---
def normalize_skill(skill):
    skill = str(skill).lower()
    skill = re.sub(r'[^a-z0-9\s]', ' ', skill)
    skill = re.sub(r'\s+', ' ', skill)
    return skill.strip()

curriculum_df['skill_normalized'] = curriculum_df['skill'].apply(normalize_skill)
job_skills_df['skill_normalized'] = job_skills_df['job_skills'].apply(normalize_skill)

# --- 2️⃣ Function to get filtered job skills for a role ---
def get_role_skills(df, role_name):
    role_name_lower = role_name.lower()
    if 'role_category' in df.columns:
        role_df = df[df['role_category'].str.lower() == role_name_lower]
    else:
        role_df = df[df['role'].str.lower().str.contains(role_name_lower, na=False)]

    skills = role_df['skill_normalized'].dropna().unique().tolist()
    return list(set(skills))  # remove duplicates

# --- 3️⃣ Extract skills for each role ---
data_scientist_skills = get_role_skills(job_skills_df, 'Data Scientist')
data_analyst_skills = get_role_skills(job_skills_df, 'Data Analyst')
business_analyst_skills = get_role_skills(job_skills_df, 'Business Analyst')

print(f"✅ Data Scientist skills collected: {len(data_scientist_skills)}")
print(f"✅ Data Analyst skills collected: {len(data_analyst_skills)}")
print(f"✅ Business Analyst skills collected: {len(business_analyst_skills)}")

# --- 4️⃣ Create combined vocabulary ---
all_skills = sorted(set(
    list(curriculum_df['skill_normalized'].unique()) +
    data_scientist_skills + data_analyst_skills + business_analyst_skills
))

# --- 5️⃣ Build curriculum vectors for universities ---
universities = curriculum_df['university'].unique()
university_vectors = {}

for uni in universities:
    uni_skills = curriculum_df[curriculum_df['university'] == uni]['skill_normalized'].tolist()
    uni_vector = np.array([1 if skill in uni_skills else 0 for skill in all_skills])
    university_vectors[uni] = uni_vector

# --- 6️⃣ Function to compute similarity for a given role ---
def compute_similarity(role_skills, role_name):
    role_vector = np.array([1 if skill in role_skills else 0 for skill in all_skills])

    scores = {}
    for uni, uni_vector in university_vectors.items():
        sim = cosine_similarity([uni_vector], [role_vector])[0][0]
        scores[uni] = sim

    df = pd.DataFrame(list(scores.items()), columns=['University', f'Similarity_with_{role_name.replace(" ", "_")}'])
    df = df.sort_values(by=df.columns[1], ascending=False)

    # Save to CSV
    csv_name = f"{role_name.replace(' ', '_')}_Curriculum_Alignment.csv"
    df.to_csv(csv_name, index=False)

    print(f"\n=== Curriculum vs {role_name} Skill Alignment ===")
    print(df)
    print(f"✅ Saved as '{csv_name}'")

    return df

# --- 7️⃣ Run for all three roles ---
df_ds = compute_similarity(data_scientist_skills, 'Data Scientist')
df_da = compute_similarity(data_analyst_skills, 'Data Analyst')
df_ba = compute_similarity(business_analyst_skills, 'Business Analyst')

# --- 8️⃣ Combine for summary ---
summary_df = df_ds.merge(df_da, on='University').merge(df_ba, on='University')
summary_df.to_csv("AllRoles_Curriculum_Alignment.csv", index=False)

print("\n✅ Combined alignment saved as 'AllRoles_Curriculum_Alignment.csv'")

# --- 9️⃣ Optional Visualization ---
summary_df.plot(
    x='University',
    kind='bar',
    figsize=(12,6),
    title='Curriculum Alignment with Job Roles',
    color=['#40be95ff', '#0072B2', '#D55E00']
)
plt.ylabel('Cosine Similarity')
plt.xticks(rotation=45, ha='right')
plt.legend(['Data Scientist', 'Data Analyst', 'Business Analyst'])
plt.tight_layout()
plt.savefig('top5us')
plt.show()


# **NON- US**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib notebook

# Set style for better-looking plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Read the CSV files
curriculum_df = pd.read_csv('skills_from_curriculum_non_us.csv')
job_skills_df = pd.read_csv('step3_job_skills_by_role.csv')

# Function to normalize skill names for better matching
def normalize_skill(skill):
    """Normalize skill names by converting to lowercase and removing special characters"""
    skill = str(skill).lower()
    skill = re.sub(r'[^a-z0-9]', ' ', skill)
    skill = re.sub(r'\s+', ' ', skill)
    return skill.strip()

# Normalize skills in both dataframes
curriculum_df['skill_normalized'] = curriculum_df['skill'].apply(normalize_skill)
job_skills_df['skill_normalized'] = job_skills_df['job_skills'].apply(normalize_skill)

# Define role categories with keyword matching patterns
# Keywords are ordered by priority (more specific first)
role_categories = {
    'Data Scientist': ['data scientist', 'ds ', 'data science'],
    'Data Analyst': ['data analyst', 'data analysis'],
    'Business Analyst': ['business analyst', 'business intelligence analyst', 'bi analyst', 'business systems analyst']
}

# Create a category column for job roles using keyword matching
def categorize_role(role):
    """
    Categorize job role based on keyword matching.
    Uses priority-based matching to handle overlapping keywords.
    """
    role_lower = role.lower()

    # Check each category's keywords
    for category, keywords in role_categories.items():
        for keyword in keywords:
            if keyword in role_lower:
                return category
    return None

job_skills_df['role_category'] = job_skills_df['role'].apply(categorize_role)

# Print categorization statistics
print("\n" + "="*80)
print("ROLE CATEGORIZATION SUMMARY")
print("="*80 + "\n")

for category in role_categories.keys():
    count = len(job_skills_df[job_skills_df['role_category'] == category])
    unique_roles = job_skills_df[job_skills_df['role_category'] == category]['role'].nunique()
    print(f"{category}:")
    print(f"  • Total records: {count}")
    print(f"  • Unique role titles: {unique_roles}")

    # Show sample role titles
    sample_roles = job_skills_df[job_skills_df['role_category'] == category]['role'].unique()[:5]
    print(f"  • Sample roles: {', '.join(sample_roles)}")
    print()

uncategorized = len(job_skills_df[job_skills_df['role_category'].isna()])
print(f"Uncategorized roles: {uncategorized}")
print(f"Total categorized: {len(job_skills_df[job_skills_df['role_category'].notna()])}")
print()

# Plot 1: Role Categorization Distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart for categorization distribution
category_counts = job_skills_df[job_skills_df['role_category'].notna()].groupby('role_category').size()
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
ax1.pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%',
        startangle=90, colors=colors, textprops={'fontsize': 11, 'weight': 'bold'})
ax1.set_title('Distribution of Job Roles by Category', fontsize=14, weight='bold', pad=20)

# Bar chart for unique role titles per category
unique_counts = []
for category in role_categories.keys():
    count = job_skills_df[job_skills_df['role_category'] == category]['role'].nunique()
    unique_counts.append(count)

ax2.bar(role_categories.keys(), unique_counts, color=colors, edgecolor='black', linewidth=1.5)
ax2.set_title('Unique Job Titles per Category', fontsize=14, weight='bold', pad=20)
ax2.set_ylabel('Number of Unique Titles', fontsize=11, weight='bold')
ax2.set_xlabel('Role Category', fontsize=11, weight='bold')
ax2.tick_params(axis='x', rotation=45)
for i, v in enumerate(unique_counts):
    ax2.text(i, v + 1, str(v), ha='center', va='bottom', fontsize=10, weight='bold')

plt.tight_layout()
plt.show()
print("📊 Plot displayed: Role Categorization Distribution\n")

# Filter to only keep categorized roles
job_skills_df = job_skills_df[job_skills_df['role_category'].notna()]

# Get all unique skills across both datasets
all_skills = sorted(set(
    list(curriculum_df['skill_normalized'].unique()) +
    list(job_skills_df['skill_normalized'].unique())
))

# Create university skill vectors
universities = curriculum_df['university'].unique()
university_vectors = {}

for uni in universities:
    uni_skills = curriculum_df[curriculum_df['university'] == uni]
    skill_vector = {}
    for skill in all_skills:
        count = uni_skills[uni_skills['skill_normalized'] == skill]['count'].sum()
        skill_vector[skill] = count
    university_vectors[uni] = skill_vector

# Create job role skill vectors (aggregated by category)
role_vectors = {}

for category in role_categories.keys():
    category_skills = job_skills_df[job_skills_df['role_category'] == category]
    skill_vector = {}
    for skill in all_skills:
        count = category_skills[category_skills['skill_normalized'] == skill]['count'].sum()
        skill_vector[skill] = count
    role_vectors[category] = skill_vector

# Convert dictionaries to matrices for cosine similarity calculation
def dict_to_vector(skill_dict, all_skills):
    """Convert skill dictionary to numpy array"""
    return np.array([skill_dict.get(skill, 0) for skill in all_skills])

# Calculate cosine similarity for each university-role combination
results = []

for role in role_categories.keys():
    role_vector = dict_to_vector(role_vectors[role], all_skills)

    print(f"\n{'='*80}")
    print(f"ALIGNMENT ANALYSIS FOR: {role}")
    print(f"{'='*80}\n")

    role_similarities = []

    for uni in universities:
        uni_vector = dict_to_vector(university_vectors[uni], all_skills)

        # Calculate cosine similarity
        similarity = cosine_similarity(
            uni_vector.reshape(1, -1),
            role_vector.reshape(1, -1)
        )[0][0]

        # Find matching skills (both have non-zero values)
        matching_skills = [
            skill for skill in all_skills
            if university_vectors[uni][skill] > 0 and role_vectors[role][skill] > 0
        ]

        # Get top matching skills by job demand
        top_skills = sorted(
            matching_skills,
            key=lambda s: role_vectors[role][s],
            reverse=True
        )[:5]

        role_similarities.append({
            'University': uni,
            'Role': role,
            'Similarity_Score': similarity * 100,
            'Matching_Skills_Count': len(matching_skills),
            'Top_Matching_Skills': ', '.join(top_skills) if top_skills else 'None'
        })

    # Sort by similarity score
    role_similarities = sorted(
        role_similarities,
        key=lambda x: x['Similarity_Score'],
        reverse=True
    )

    # Print rankings
    for rank, item in enumerate(role_similarities, 1):
        medal = '🥇' if rank == 1 else '🥈' if rank == 2 else '🥉' if rank == 3 else '  '
        print(f"{medal} Rank {rank}: {item['University']}")
        print(f"   Similarity Score: {item['Similarity_Score']:.2f}%")
        print(f"   Matching Skills: {item['Matching_Skills_Count']}")
        print(f"   Top Skills: {item['Top_Matching_Skills']}")
        print()

    results.extend(role_similarities)

# Create comprehensive results dataframe
results_df = pd.DataFrame(results)

# Save to CSV
results_df.to_csv('university_job_alignment_results.csv', index=False)
print(f"\nResults saved to 'university_job_alignment_results.csv'")

# Create summary pivot table
print(f"\n{'='*80}")
print("SUMMARY: Alignment Scores by University and Role")
print(f"{'='*80}\n")

pivot_table = results_df.pivot_table(
    values='Similarity_Score',
    index='University',
    columns='Role',
    aggfunc='first'
).round(2)

print(pivot_table)

# Plot 2: Heatmap of Similarity Scores
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, fmt='.2f', cmap='YlOrRd',
            cbar_kws={'label': 'Similarity Score (%)'},
            linewidths=2, linecolor='white',
            square=True, vmin=0, vmax=100)
plt.title('University-Role Alignment Heatmap\n(Cosine Similarity Scores)',
          fontsize=14, weight='bold', pad=20)
plt.xlabel('Role Category', fontsize=11, weight='bold')
plt.ylabel('University', fontsize=11, weight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()
print("📊 Plot displayed: Similarity Heatmap\n")

# Find best university for each role
print(f"\n{'='*80}")
print("BEST UNIVERSITY FOR EACH ROLE")
print(f"{'='*80}\n")

for role in role_categories.keys():
    best = results_df[results_df['Role'] == role].sort_values(
        'Similarity_Score',
        ascending=False
    ).iloc[0]
    print(f"{role}:")
    print(f"  🏆 {best['University']}")
    print(f"  Score: {best['Similarity_Score']:.2f}%")
    print(f"  Matching Skills: {best['Matching_Skills_Count']}")
    print()

# Plot 3: Grouped Bar Chart - Similarity Scores by Role
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(universities))
width = 0.25

for i, role in enumerate(role_categories.keys()):
    scores = [results_df[(results_df['University'] == uni) &
                         (results_df['Role'] == role)]['Similarity_Score'].values[0]
              for uni in universities]
    ax.bar(x + i*width, scores, width, label=role, edgecolor='black', linewidth=1)

ax.set_xlabel('University', fontsize=11, weight='bold')
ax.set_ylabel('Similarity Score (%)', fontsize=11, weight='bold')
ax.set_title('University-Role Alignment Comparison', fontsize=14, weight='bold', pad=20)
ax.set_xticks(x + width)
ax.set_xticklabels([uni.split('(')[0].strip() for uni in universities], rotation=45, ha='right')
ax.legend(title='Role Category', fontsize=10, title_fontsize=11)
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()
print("📊 Plot displayed: Similarity Comparison Bars\n")

# Plot 4: Best University for Each Role (Visual Summary)
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
colors_role = ['#FF6B6B', '#4ECDC4', '#45B7D1']

for idx, (role, color) in enumerate(zip(role_categories.keys(), colors_role)):
    role_data = results_df[results_df['Role'] == role].sort_values('Similarity_Score', ascending=True)

    axes[idx].barh(range(len(role_data)), role_data['Similarity_Score'],
                   color=color, edgecolor='black', linewidth=1.5)
    axes[idx].set_yticks(range(len(role_data)))
    axes[idx].set_yticklabels([uni.split('(')[0].strip() for uni in role_data['University']], fontsize=9)
    axes[idx].set_xlabel('Similarity Score (%)', fontsize=10, weight='bold')
    axes[idx].set_title(f'{role}', fontsize=12, weight='bold', pad=10)
    axes[idx].grid(axis='x', alpha=0.3)

    # Add value labels
    for i, v in enumerate(role_data['Similarity_Score']):
        axes[idx].text(v + 0.5, i, f'{v:.1f}%', va='center', fontsize=9, weight='bold')

plt.suptitle('University Rankings by Role Category', fontsize=16, weight='bold', y=1.02)
plt.tight_layout()
plt.show()
print("📊 Plot displayed: Rankings by Role\n")

# Create a detailed skills comparison
print(f"\n{'='*80}")
print("DETAILED SKILLS ANALYSIS")
print(f"{'='*80}\n")

for role in role_categories.keys():
    print(f"\n{role} - Top 10 Required Skills:")
    role_skills = sorted(
        [(skill, role_vectors[role][skill]) for skill in all_skills if role_vectors[role][skill] > 0],
        key=lambda x: x[1],
        reverse=True
    )[:10]

    for skill, count in role_skills:
        print(f"  • {skill}: {count} job postings")

    # Show which universities teach these top skills
    print(f"\n  Universities teaching these skills:")
    for uni in universities:
        taught_count = sum(1 for skill, _ in role_skills if university_vectors[uni][skill] > 0)
        coverage = (taught_count / len(role_skills)) * 100
        print(f"    {uni}: {taught_count}/{len(role_skills)} skills ({coverage:.1f}% coverage)")
    print()

# Plot 5: Skills Coverage Analysis
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, role in enumerate(role_categories.keys()):
    # Get top 10 skills for this role
    role_skills = sorted(
        [(skill, role_vectors[role][skill]) for skill in all_skills if role_vectors[role][skill] > 0],
        key=lambda x: x[1],
        reverse=True
    )[:10]

    # Calculate coverage for each university
    coverage_data = []
    for uni in universities:
        taught_count = sum(1 for skill, _ in role_skills if university_vectors[uni][skill] > 0)
        coverage_pct = (taught_count / len(role_skills)) * 100
        coverage_data.append(coverage_pct)

    axes[idx].bar(range(len(universities)), coverage_data,
                  color=colors_role[idx], edgecolor='black', linewidth=1.5)
    axes[idx].set_xticks(range(len(universities)))
    axes[idx].set_xticklabels([uni.split('(')[0].strip() for uni in universities],
                               rotation=45, ha='right', fontsize=9)
    axes[idx].set_ylabel('Coverage (%)', fontsize=10, weight='bold')
    axes[idx].set_title(f'{role}\nTop 10 Skills Coverage', fontsize=11, weight='bold')
    axes[idx].set_ylim(0, 100)
    axes[idx].grid(axis='y', alpha=0.3)

    # Add value labels
    for i, v in enumerate(coverage_data):
        axes[idx].text(i, v + 2, f'{v:.0f}%', ha='center', va='bottom', fontsize=9, weight='bold')

plt.suptitle('University Coverage of Top 10 In-Demand Skills by Role',
             fontsize=14, weight='bold', y=1.02)
plt.tight_layout()
plt.show()
print("📊 Plot displayed: Skills Coverage Analysis\n")

# Plot 6: Matching Skills Count Comparison
fig, ax = plt.subplots(figsize=(12, 6))
matching_pivot = results_df.pivot_table(
    values='Matching_Skills_Count',
    index='University',
    columns='Role',
    aggfunc='first'
)

matching_pivot.plot(kind='bar', ax=ax, edgecolor='black', linewidth=1.5)
ax.set_xlabel('University', fontsize=11, weight='bold')
ax.set_ylabel('Number of Matching Skills', fontsize=11, weight='bold')
ax.set_title('Number of Matching Skills Between Curriculum and Job Requirements',
             fontsize=14, weight='bold', pad=20)
ax.set_xticklabels([uni.split('(')[0].strip() for uni in matching_pivot.index],
                    rotation=45, ha='right')
ax.legend(title='Role Category', fontsize=10, title_fontsize=11)
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()
print("📊 Plot displayed: Matching Skills Comparison\n")

# Plot 7: Radar Chart for Overall Alignment
from math import pi

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

angles = [n / float(len(role_categories)) * 2 * pi for n in range(len(role_categories))]
angles += angles[:1]

for uni in universities:
    values = []
    for role in role_categories.keys():
        score = results_df[(results_df['University'] == uni) &
                          (results_df['Role'] == role)]['Similarity_Score'].values[0]
        values.append(score)
    values += values[:1]

    ax.plot(angles, values, 'o-', linewidth=2, label=uni.split('(')[0].strip())
    ax.fill(angles, values, alpha=0.15)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(role_categories.keys(), fontsize=11, weight='bold')
ax.set_ylim(0, 100)
ax.set_ylabel('Similarity Score (%)', fontsize=11, weight='bold', labelpad=30)
ax.set_title('Overall University-Role Alignment Profile',
             fontsize=14, weight='bold', pad=30)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=10)
ax.grid(True)
plt.tight_layout()
plt.show()
print("📊 Plot displayed: Alignment Radar Chart\n")

print("\n" + "="*80)
print("ALL VISUALIZATIONS DISPLAYED!")
print("="*80)
print("\n📊 Displayed plots:")
print("  1. Role Categorization Distribution - Role distribution and unique titles")
print("  2. Similarity Heatmap - Heatmap of all similarity scores")
print("  3. Similarity Comparison Bars - Grouped bar chart comparison")
print("  4. Rankings by Role - Horizontal bar charts showing rankings")
print("  5. Skills Coverage Analysis - Top 10 skills coverage by university")
print("  6. Matching Skills Comparison - Number of matching skills")
print("  7. Alignment Radar Chart - Overall alignment profile radar chart")
print("\n")

In [ ]:
# --- Curriculum vs Job Skill Alignment for All Roles ---

import pandas as pd
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# --- Load data ---
curriculum_df = pd.read_csv('skills_from_curriculum_non_us.csv')
job_skills_df = pd.read_csv('step3_job_skills_by_role.csv')

# --- 1️⃣ Normalize skill names ---
def normalize_skill(skill):
    skill = str(skill).lower()
    skill = re.sub(r'[^a-z0-9\s]', ' ', skill)
    skill = re.sub(r'\s+', ' ', skill)
    return skill.strip()

curriculum_df['skill_normalized'] = curriculum_df['skill'].apply(normalize_skill)
job_skills_df['skill_normalized'] = job_skills_df['job_skills'].apply(normalize_skill)

# --- 2️⃣ Function to get filtered job skills for a role ---
def get_role_skills(df, role_name):
    role_name_lower = role_name.lower()
    if 'role_category' in df.columns:
        role_df = df[df['role_category'].str.lower() == role_name_lower]
    else:
        role_df = df[df['role'].str.lower().str.contains(role_name_lower, na=False)]

    skills = role_df['skill_normalized'].dropna().unique().tolist()
    return list(set(skills))  # remove duplicates

# --- 3️⃣ Extract skills for each role ---
data_scientist_skills = get_role_skills(job_skills_df, 'Data Scientist')
data_analyst_skills = get_role_skills(job_skills_df, 'Data Analyst')
business_analyst_skills = get_role_skills(job_skills_df, 'Business Analyst')

print(f"✅ Data Scientist skills collected: {len(data_scientist_skills)}")
print(f"✅ Data Analyst skills collected: {len(data_analyst_skills)}")
print(f"✅ Business Analyst skills collected: {len(business_analyst_skills)}")

# --- 4️⃣ Create combined vocabulary ---
all_skills = sorted(set(
    list(curriculum_df['skill_normalized'].unique()) +
    data_scientist_skills + data_analyst_skills + business_analyst_skills
))

# --- 5️⃣ Build curriculum vectors for universities ---
universities = curriculum_df['university'].unique()
university_vectors = {}

for uni in universities:
    uni_skills = curriculum_df[curriculum_df['university'] == uni]['skill_normalized'].tolist()
    uni_vector = np.array([1 if skill in uni_skills else 0 for skill in all_skills])
    university_vectors[uni] = uni_vector

# --- 6️⃣ Function to compute similarity for a given role ---
def compute_similarity(role_skills, role_name):
    role_vector = np.array([1 if skill in role_skills else 0 for skill in all_skills])

    scores = {}
    for uni, uni_vector in university_vectors.items():
        sim = cosine_similarity([uni_vector], [role_vector])[0][0]
        scores[uni] = sim

    df = pd.DataFrame(list(scores.items()), columns=['University', f'Similarity_with_{role_name.replace(" ", "_")}'])
    df = df.sort_values(by=df.columns[1], ascending=False)

    # Save to CSV
    csv_name = f"{role_name.replace(' ', '_')}_Curriculum_Alignment.csv"
    df.to_csv(csv_name, index=False)

    print(f"\n=== Curriculum vs {role_name} Skill Alignment ===")
    print(df)
    print(f"✅ Saved as '{csv_name}'")

    return df

# --- 7️⃣ Run for all three roles ---
df_ds = compute_similarity(data_scientist_skills, 'Data Scientist')
df_da = compute_similarity(data_analyst_skills, 'Data Analyst')
df_ba = compute_similarity(business_analyst_skills, 'Business Analyst')

# --- 8️⃣ Combine for summary ---
summary_df = df_ds.merge(df_da, on='University').merge(df_ba, on='University')
summary_df.to_csv("AllRoles_Curriculum_Alignment.csv", index=False)

print("\n✅ Combined alignment saved as 'AllRoles_Curriculum_Alignment.csv'")

# --- 9️⃣ Optional Visualization ---
summary_df.plot(
    x='University',
    kind='bar',
    figsize=(12,6),
    title='Curriculum Alignment with Job Roles',
    color=['#2ea597', '#688ae8', '#8456ce']
)
plt.ylabel('Cosine Similarity')
plt.xticks(rotation=45, ha='right')
plt.legend(['Data Scientist', 'Data Analyst', 'Business Analyst'])
plt.tight_layout()
plt.savefig('top5nonus.png')
plt.show()


# **TOP 10 US**

In [ ]:
# --- Curriculum vs Job Skill Alignment for All Roles ---

import pandas as pd
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# --- Load data ---
curriculum_df = pd.read_csv('skills_from_curriculum (1).csv')
job_skills_df = pd.read_csv('step3_job_skills_by_role.csv')

# --- 1️⃣ Normalize skill names ---
def normalize_skill(skill):
    skill = str(skill).lower()
    skill = re.sub(r'[^a-z0-9\s]', ' ', skill)
    skill = re.sub(r'\s+', ' ', skill)
    return skill.strip()

curriculum_df['skill_normalized'] = curriculum_df['skill'].apply(normalize_skill)
job_skills_df['skill_normalized'] = job_skills_df['job_skills'].apply(normalize_skill)

# --- 2️⃣ Function to get filtered job skills for a role ---
def get_role_skills(df, role_name):
    role_name_lower = role_name.lower()
    if 'role_category' in df.columns:
        role_df = df[df['role_category'].str.lower() == role_name_lower]
    else:
        role_df = df[df['role'].str.lower().str.contains(role_name_lower, na=False)]

    skills = role_df['skill_normalized'].dropna().unique().tolist()
    return list(set(skills))  # remove duplicates

# --- 3️⃣ Extract skills for each role ---
data_scientist_skills = get_role_skills(job_skills_df, 'Data Scientist')
data_analyst_skills = get_role_skills(job_skills_df, 'Data Analyst')
business_analyst_skills = get_role_skills(job_skills_df, 'Business Analyst')

print(f"✅ Data Scientist skills collected: {len(data_scientist_skills)}")
print(f"✅ Data Analyst skills collected: {len(data_analyst_skills)}")
print(f"✅ Business Analyst skills collected: {len(business_analyst_skills)}")

# --- 4️⃣ Create combined vocabulary ---
all_skills = sorted(set(
    list(curriculum_df['skill_normalized'].unique()) +
    data_scientist_skills + data_analyst_skills + business_analyst_skills
))

# --- 5️⃣ Build curriculum vectors for universities ---
universities = curriculum_df['university'].unique()
university_vectors = {}

for uni in universities:
    uni_skills = curriculum_df[curriculum_df['university'] == uni]['skill_normalized'].tolist()
    uni_vector = np.array([1 if skill in uni_skills else 0 for skill in all_skills])
    university_vectors[uni] = uni_vector

# --- 6️⃣ Function to compute similarity for a given role ---
def compute_similarity(role_skills, role_name):
    role_vector = np.array([1 if skill in role_skills else 0 for skill in all_skills])

    scores = {}
    for uni, uni_vector in university_vectors.items():
        sim = cosine_similarity([uni_vector], [role_vector])[0][0]
        scores[uni] = sim

    df = pd.DataFrame(list(scores.items()), columns=['University', f'Similarity_with_{role_name.replace(" ", "_")}'])
    df = df.sort_values(by=df.columns[1], ascending=False)

    # Save to CSV
    csv_name = f"{role_name.replace(' ', '_')}_Curriculum_Alignment.csv"
    df.to_csv(csv_name, index=False)

    print(f"\n=== Curriculum vs {role_name} Skill Alignment ===")
    print(df)
    print(f"✅ Saved as '{csv_name}'")

    return df

# --- 7️⃣ Run for all three roles ---
df_ds = compute_similarity(data_scientist_skills, 'Data Scientist')
df_da = compute_similarity(data_analyst_skills, 'Data Analyst')
df_ba = compute_similarity(business_analyst_skills, 'Business Analyst')

# --- 8️⃣ Combine for summary ---
summary_df = df_ds.merge(df_da, on='University').merge(df_ba, on='University')
summary_df.to_csv("AllRoles_Curriculum_Alignment.csv", index=False)

print("\n✅ Combined alignment saved as 'AllRoles_Curriculum_Alignment.csv'")

# --- 9️⃣ Optional Visualization ---
summary_df.plot(
    x='University',
    kind='bar',
    figsize=(12,6),
    title='Curriculum Alignment with Job Roles',
    color=['#4C72B0', '#55A868', '#C44E52']
)
plt.ylabel('Cosine Similarity')
plt.legend(['Data Scientist', 'Data Analyst', 'Business Analyst'])
plt.tight_layout()

plt.savefig('top10us.png')


plt.show()


**Employability Index Calculation**

In [ ]:
# --- Composite Employability Index Calculation ---

import pandas as pd
import numpy as np

# Load your combined alignment file (already created previously)
alignment_df = pd.read_csv("AllRoles_Curriculum_Alignment.csv")

# Load the curriculum and job skills data again
curriculum_df = pd.read_csv("skills_from_curriculum (1).csv")
job_skills_df = pd.read_csv("step3_job_skills_by_role.csv")

# Normalize skill names
import re
def normalize_skill(skill):
    skill = str(skill).lower()
    skill = re.sub(r'[^a-z0-9\s]', ' ', skill)
    skill = re.sub(r'\s+', ' ', skill)
    return skill.strip()

curriculum_df['skill_normalized'] = curriculum_df['skill'].apply(normalize_skill)
job_skills_df['skill_normalized'] = job_skills_df['job_skills'].apply(normalize_skill)

# --- Step 1️⃣: Combine all job skills and count frequency ---
skill_demand = job_skills_df['skill_normalized'].value_counts().reset_index()
skill_demand.columns = ['skill', 'count']

# --- Step 2️⃣: Compute Coverage Ratio and Weighted Demand Match ---
universities = curriculum_df['university'].unique()

coverage_data = []

for uni in universities:
    uni_skills = set(curriculum_df[curriculum_df['university'] == uni]['skill_normalized'])

    # Total job skills
    total_job_skills = set(skill_demand['skill'])

    # Coverage ratio (how many job skills the university teaches)
    coverage_ratio = len(uni_skills.intersection(total_job_skills)) / len(total_job_skills)

    # Weighted demand match (how many high-demand skills are covered, weighted by frequency)
    matched_skills = skill_demand[skill_demand['skill'].isin(uni_skills)]
    weighted_match = matched_skills['count'].sum() / skill_demand['count'].sum()

    coverage_data.append([uni, coverage_ratio, weighted_match])

coverage_df = pd.DataFrame(coverage_data, columns=['University', 'Coverage_Ratio', 'Weighted_Match'])

# --- Step 3️⃣: Merge with similarity scores ---
merged = alignment_df.merge(coverage_df, on='University', how='left')

# Compute average similarity across all three roles
merged['Avg_Similarity'] = merged[['Similarity_with_Data_Scientist',
                                   'Similarity_with_Data_Analyst',
                                   'Similarity_with_Business_Analyst']].mean(axis=1)

# --- Step 4️⃣: Calculate Employability Index ---
merged['Employability_Index'] = (
    0.5 * merged['Avg_Similarity'] +
    0.3 * merged['Coverage_Ratio'] +
    0.2 * merged['Weighted_Match']
)

# --- Step 5️⃣: Sort and Display ---
merged = merged.sort_values('Employability_Index', ascending=False)
print("\n=== Composite Employability Index (Higher = Better Alignment) ===")
print(merged[['University', 'Avg_Similarity', 'Coverage_Ratio', 'Weighted_Match', 'Employability_Index']])

# --- Step 6️⃣: Save to CSV ---
merged.to_csv("Composite_Employability_Index.csv", index=False)
print("\n✅ Saved as 'Composite_Employability_Index.csv'")

# --- Step 7️⃣: Visualization ---
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
plt.barh(merged['University'], merged['Employability_Index'], color='teal')
plt.xlabel("Employability Index Score")
plt.title("Composite Employability Index by University")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


# **Topic Modelling using LDA**
**Heatmap Analysis**

In [ ]:
# --- 🔹 Step 1: Imports ---
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import seaborn as sns

# --- 🔹 Step 2: Load your datasets ---
curr_df = pd.read_csv("skills_from_curriculum (1).csv")
jobs_df = pd.read_csv("step3_job_skills_by_role.csv")

# Label sources
curr_df["source"] = "Curriculum"
jobs_df["source"] = "Job"

# --- 🔹 Step 3: Combine into one dataframe ---
combined_df = pd.concat([
    curr_df[["university", "skill", "source"]],
    jobs_df.assign(university="Industry")[["university", "job_skills", "source"]].rename(columns={"job_skills": "skill"})
], ignore_index=True)

# Clean text
combined_df["skill"] = combined_df["skill"].fillna("").str.lower()

# --- 🔹 Step 4: Convert skills to Bag-of-Words matrix ---
vectorizer = CountVectorizer(stop_words="english")
X = vectorizer.fit_transform(combined_df["skill"])
words = np.array(vectorizer.get_feature_names_out())

print(f"✅ Vocabulary size: {len(words)}")

# --- 🔹 Step 5: Fit LDA ---
lda = LatentDirichletAllocation(
    n_components=12,       # try 8–10 for balanced detail
    random_state=42,
    learning_method="batch",
    max_iter=30
)
lda.fit(X)

# --- 🔹 Step 6: Extract top words per topic ---
def get_top_words(model, feature_names, n_top_words=10):
    topics = {}
    for topic_idx, topic in enumerate(model.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        topics[f"Topic_{topic_idx+1}"] = top_words
    return topics

topics_dict = get_top_words(lda, words)
print("\n=== 🔹 Top Words per Topic ===")
for t, w in topics_dict.items():
    print(f"{t}: {', '.join(w)}")

# --- 🔹 Step 7: Create topic-word probability DataFrame ---
topic_word_matrix = lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis]
topic_word_df = pd.DataFrame(topic_word_matrix.T, columns=[f"Topic_{i+1}" for i in range(lda.n_components)])
topic_word_df.insert(0, "word", words)
topic_word_df.to_csv("LDA_Topic_Word_Matrix.csv", index=False)

# --- 🔹 Step 8: Get skill-topic probabilities ---
doc_topic_matrix = lda.transform(X)
topic_cols = [f"Topic_{i+1}" for i in range(lda.n_components)]
doc_topic_df = pd.DataFrame(doc_topic_matrix, columns=topic_cols)

combined_topic_df = pd.concat([combined_df.reset_index(drop=True), doc_topic_df], axis=1)
combined_topic_df.to_csv("LDA_Skill_Topic_Assignment.csv", index=False)
print("✅ Saved: 'LDA_Skill_Topic_Assignment.csv'")

# --- 🔹 Step 9: Average topic weights by source (Curriculum vs Jobs) ---
avg_by_source = combined_topic_df.groupby("source")[topic_cols].mean().T
avg_by_source["Difference"] = avg_by_source["Job"] - avg_by_source["Curriculum"]
avg_by_source = avg_by_source.sort_values("Difference", ascending=False)
print("\n=== 🔹 Topic Weights by Source ===")
print(avg_by_source.round(3))

# --- 🔹 Step 10: University-wise topic alignment ---
uni_topic = combined_topic_df.groupby("university")[topic_cols].mean()
uni_topic = uni_topic.sort_index()

# Normalize each university’s topic weights (optional)
uni_topic_norm = uni_topic.div(uni_topic.sum(axis=1), axis=0)
uni_topic_norm.to_csv("University_Topic_Alignment.csv")
print("\n✅ Saved: 'University_Topic_Alignment.csv'")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1️⃣ Replace this with your real data (10 universities × 8 topics) ---
data = {
    'University': [
        'Carnegie Mellon (Tepper)',
        'MIT (Sloan)',
        'Columbia University',
        'New York University (Stern)',
        'UT Austin (McCombs)',
        'UCLA (Anderson)',
        'USC (Marshall)',
        'Northwestern (Kellogg)',
        'University of Chicago (Booth)',
        'Duke University (Fuqua)'
    ],
    'Topic_1': [0.134, 0.102, 0.110, 0.125, 0.118, 0.107, 0.120, 0.111, 0.106, 0.112],
    'Topic_2': [0.163, 0.148, 0.150, 0.158, 0.160, 0.153, 0.155, 0.150, 0.157, 0.149],
    'Topic_3': [0.176, 0.160, 0.155, 0.164, 0.170, 0.158, 0.162, 0.166, 0.168, 0.159],
    'Topic_4': [0.092, 0.081, 0.078, 0.090, 0.088, 0.080, 0.083, 0.084, 0.085, 0.081],
    'Topic_5': [0.075, 0.085, 0.079, 0.080, 0.082, 0.078, 0.079, 0.081, 0.083, 0.080],
    'Topic_6': [0.125, 0.122, 0.118, 0.130, 0.122, 0.119, 0.120, 0.124, 0.126, 0.121],
    'Topic_7': [0.115, 0.108, 0.105, 0.110, 0.112, 0.106, 0.107, 0.108, 0.111, 0.109],
    'Topic_8': [0.121, 0.119, 0.117, 0.123, 0.125, 0.118, 0.119, 0.120, 0.124, 0.122]
}

df = pd.DataFrame(data)
df.set_index('University', inplace=True)

# --- 2️⃣ Optional: Rename topics to meaningful labels ---
df.columns = [
    'Optimization & Stats',
    'Core ML & Modeling',
    'Applied ML Workflows',
    'NLP & Cloud Processing',
    'Analytics Tools',
    'Forecasting & Big Data',
    'Cloud & Programming',
    'Programming & Databases'
]

# --- 3️⃣ Absolute Heatmap (Actual Topic Weights) ---
plt.figure(figsize=(13,7))
sns.heatmap(df, annot=True, fmt=".3f", cmap="YlGnBu", linewidths=0.5, cbar_kws={'label': 'Topic Weight'})
plt.title("Curriculum Topic Distribution Across 10 Universities", fontsize=15, fontweight='bold', pad=20)
plt.xlabel("Skill Topic Cluster", fontsize=12, fontweight='bold')
plt.ylabel("University", fontsize=12, fontweight='bold')
plt.xticks(rotation=30, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('heatmap')
plt.show()

# --- 4️⃣ Relative Difference Heatmap (Highlight Gaps vs Average) ---
df_diff = df - df.mean()

plt.figure(figsize=(13,7))
sns.heatmap(df_diff, annot=True, fmt=".3f", cmap="RdYlGn", center=0, linewidths=0.5,
            cbar_kws={'label': 'Deviation from Average'})
plt.title("University Topic Emphasis vs. Peer Average", fontsize=15, fontweight='bold', pad=20)
plt.xlabel("Skill Topic Cluster", fontsize=12, fontweight='bold')
plt.ylabel("University", fontsize=12, fontweight='bold')
plt.xticks(rotation=30, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()
